# Market Basket Analysis

Dataset: Groceries dataset for Market Basket Analysis (MBA)

Downloaded from: https://www.kaggle.com/datasets/rashikrahmanpritom/groceries-dataset-for-market-basket-analysismba

Used: "Groceries data.csv" which requires some pre-processing before apriori() and association_rules()

## 1. Import Libraries & Load Dataset

In [109]:
# Import libraries and packages

# For data transformation and manipulation
import pandas as pd

# For data visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# For market basket analysis
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [110]:
# Load the dataset
groceries = pd.read_csv("Groceries data.csv")

In [111]:
# View the first 10 records of the dataset
groceries.head(10)

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,1808,2015-07-21,tropical fruit,2015,7,21,1
1,2552,2015-05-01,whole milk,2015,5,1,4
2,2300,2015-09-19,pip fruit,2015,9,19,5
3,1187,2015-12-12,other vegetables,2015,12,12,5
4,3037,2015-01-02,whole milk,2015,1,2,4
5,4941,2015-02-14,rolls/buns,2015,2,14,5
6,4501,2015-08-05,other vegetables,2015,8,5,2
7,3803,2015-12-23,pot plants,2015,12,23,2
8,2762,2015-03-20,whole milk,2015,3,20,4
9,4119,2015-12-02,tropical fruit,2015,12,2,2


In [112]:
# View the dataset info
groceries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
 3   year             38765 non-null  int64 
 4   month            38765 non-null  int64 
 5   day              38765 non-null  int64 
 6   day_of_week      38765 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 2.1+ MB


In [113]:
# Visualise item frequency
plt.figure(figsize = (200, 40))

sns.countplot(data = groceries,
              x = "itemDescription",
              order = groceries["itemDescription"].value_counts().index)

plt.xticks(rotation = 45)

plt.show()

## 2. Prepare the Dataset

In [114]:
# Convert the Date column to datetime type (optional)
groceries["Date"] = pd.to_datetime(groceries["Date"])

# Check the results
groceries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Member_number    38765 non-null  int64         
 1   Date             38765 non-null  datetime64[ns]
 2   itemDescription  38765 non-null  object        
 3   year             38765 non-null  int64         
 4   month            38765 non-null  int64         
 5   day              38765 non-null  int64         
 6   day_of_week      38765 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 2.1+ MB


In [115]:
# Rename the columns for convenience (optional)
groceries = groceries.rename(columns = {"Member_number": "member_id", "Date": "date", "itemDescription": "items"})

# Check the results
groceries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   member_id    38765 non-null  int64         
 1   date         38765 non-null  datetime64[ns]
 2   items        38765 non-null  object        
 3   year         38765 non-null  int64         
 4   month        38765 non-null  int64         
 5   day          38765 non-null  int64         
 6   day_of_week  38765 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 2.1+ MB


In [130]:
# Prepare the dataset for onehot encoding
groceries_grouped = groceries.groupby(["member_id", "date"])["items"].apply(list).reset_index()

# Check the results
groceries_grouped.head(10)

,member_id,date,items
0,1000,2014-06-24,"[whole milk, pastry, salty snack]"
1,1000,2015-03-15,"[sausage, whole milk, semi-finished bread, yog..."
2,1000,2015-05-27,"[soda, pickled vegetables]"
3,1000,2015-07-24,"[canned beer, misc. beverages]"
4,1000,2015-11-25,"[sausage, hygiene articles]"
5,1001,2014-07-02,"[sausage, whole milk, rolls/buns]"
6,1001,2014-12-12,"[whole milk, soda]"
7,1001,2015-01-20,"[frankfurter, soda, whipped/sour cream]"
8,1001,2015-02-05,"[frankfurter, curd]"
9,1001,2015-04-14,"[beef, white bread]"


In [131]:
# Onehot encode the items

# Convert column "items" into a list of list
itemsets = groceries_grouped["items"].tolist()

# Instantiate the encoder
encoder = TransactionEncoder().fit(itemsets)

# Use the encoder to transform itemsets
itemsets_encoded = encoder.transform(itemsets)

# Convert itemsets into a df
itemsets_encoded_df = pd.DataFrame(itemsets_encoded, columns = encoder.columns_)

# Check the results
itemsets_encoded_df.head(10)


,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False


## 3. Apply Apriori Algorithm

In [133]:
# Find frequent itemsets

# Apply apriori algorithm
frequent_itemsets = apriori(itemsets_encoded_df, min_support = 0.001, use_colnames = True)

# Check the results
frequent_itemsets

,support,itemsets
0,0.004010,(Instant food products)
1,0.021386,(UHT-milk)
2,0.001470,(abrasive cleaner)
3,0.001938,(artif. sweetener)
4,0.008087,(baking powder)
...,...,...
745,0.001136,"(rolls/buns, whole milk, sausage)"
746,0.001002,"(soda, rolls/buns, whole milk)"
747,0.001337,"(rolls/buns, whole milk, yogurt)"
748,0.001069,"(soda, whole milk, sausage)"


## 4. Derive Association Rules

In [134]:
# Find rules for frequent itemsets

# Apply association_rules
rules = association_rules(frequent_itemsets, metric = "lift", min_threshold = 0.5, num_itemsets = len(frequent_itemsets))

# Check the results
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(bottled water),(UHT-milk),0.060683,0.021386,0.001069,0.017621,0.823954,1.0,-0.000228,0.996168,-0.185312,0.013201,-0.003847,0.033811
1,(UHT-milk),(bottled water),0.021386,0.060683,0.001069,0.050000,0.823954,1.0,-0.000228,0.988755,-0.179204,0.013201,-0.011373,0.033811
2,(other vegetables),(UHT-milk),0.122101,0.021386,0.002139,0.017515,0.818993,1.0,-0.000473,0.996060,-0.201119,0.015130,-0.003956,0.058758
3,(UHT-milk),(other vegetables),0.021386,0.122101,0.002139,0.100000,0.818993,1.0,-0.000473,0.975443,-0.184234,0.015130,-0.025175,0.058758
4,(rolls/buns),(UHT-milk),0.110005,0.021386,0.001804,0.016403,0.767013,1.0,-0.000548,0.994934,-0.254457,0.013925,-0.005092,0.050389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,"(whole milk, yogurt)",(sausage),0.011161,0.060349,0.001470,0.131737,2.182917,1.0,0.000797,1.082219,0.548014,0.020992,0.075973,0.078050
1228,"(sausage, yogurt)",(whole milk),0.005748,0.157923,0.001470,0.255814,1.619866,1.0,0.000563,1.131541,0.384877,0.009065,0.116250,0.132562
1229,(whole milk),"(sausage, yogurt)",0.157923,0.005748,0.001470,0.009310,1.619866,1.0,0.000563,1.003596,0.454430,0.009065,0.003583,0.132562
1230,(sausage),"(whole milk, yogurt)",0.060349,0.011161,0.001470,0.024363,2.182917,1.0,0.000797,1.013532,0.576701,0.020992,0.013351,0.078050


In [138]:
# Filter the rules
strong_rules = rules[(rules["confidence"] >= 0.01) & (rules["lift"] >= 1.0)]

# Check the results
strong_rules.sort_values(["confidence", "lift"], ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
1228,"(sausage, yogurt)",(whole milk),0.005748,0.157923,0.001470,0.255814,1.619866,1.0,0.000563,1.131541,0.384877,0.009065,0.116250,0.132562
1203,"(rolls/buns, sausage)",(whole milk),0.005347,0.157923,0.001136,0.212500,1.345594,1.0,0.000292,1.069304,0.258214,0.007007,0.064813,0.109847
1221,"(soda, sausage)",(whole milk),0.005948,0.157923,0.001069,0.179775,1.138374,1.0,0.000130,1.026642,0.122281,0.006568,0.025951,0.093273
1102,(semi-finished bread),(whole milk),0.009490,0.157923,0.001671,0.176056,1.114825,1.0,0.000172,1.022008,0.103985,0.010081,0.021534,0.093318
1215,"(rolls/buns, yogurt)",(whole milk),0.007819,0.157923,0.001337,0.170940,1.082428,1.0,0.000102,1.015701,0.076751,0.008130,0.015459,0.089702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223,(soda),"(whole milk, sausage)",0.097106,0.008955,0.001069,0.011012,1.229612,1.0,0.000200,1.002079,0.206819,0.010185,0.002075,0.065207
950,(rolls/buns),(packaged fruit/vegetables),0.110005,0.008488,0.001203,0.010936,1.288421,1.0,0.000269,1.002475,0.251525,0.010256,0.002469,0.076334
1103,(whole milk),(semi-finished bread),0.157923,0.009490,0.001671,0.010580,1.114825,1.0,0.000172,1.001101,0.122314,0.010081,0.001100,0.093318
1205,(rolls/buns),"(whole milk, sausage)",0.110005,0.008955,0.001136,0.010328,1.153275,1.0,0.000151,1.001387,0.149331,0.009643,0.001385,0.068597
